In [6]:
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage import exposure
import matplotlib.image as mpimg
from skimage import filters
from scipy.signal import convolve2d
import cv2
from scipy.ndimage import median_filter
from skimage import exposure, img_as_ubyte
from PIL import Image
from tqdm import tqdm

In [7]:
path_im = 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\univesita\\Elaborazione immagini biomediche\\RETINA\\Original'
path_GT = 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\univesita\\Elaborazione immagini biomediche\\RETINA\\Ground truth'
path_result = 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\univesita\\Elaborazione immagini biomediche\\RETINA\\Preprocess'
images = os.listdir(path_im)
GT = os.listdir(path_GT)

if len(images) != len(GT):
    print('errore manca qualcosa le liste sono di lunghezze diversa')


In [8]:
def equalize_within_mask(image, mask, range_min=0, range_max=1):
    # Applica la maschera all'immagine
    masked_image = np.copy(image)
    masked_image[~mask] = 0

    # Calcola l'istogramma dell'immagine all'interno della maschera
    hist, bin_centers = exposure.histogram(masked_image)

    # Equalizza l'istogramma all'interno del range specificato
    equalized_image = exposure.rescale_intensity(masked_image, in_range='image', out_range = (range_min,range_max))

    image_to_add = np.copy(image)
    image_to_add[mask] = 0
    equalized_image = equalized_image + image_to_add

    return equalized_image



def convolution_with_max_density_optic(array, kernel_list):
    max_density_optic = float('-inf')
    result = None

    for kernel in kernel_list:
        conv_result = array *2* kernel
        density_optic = np.sum(conv_result)

        if density_optic > max_density_optic:
            max_density_optic = density_optic
            if np.array_equal(kernel,kernel_list[-1]):
                result = np.zeros(np.shape(kernel))
            else:
                result = conv_result


    return result

def low_pass(img,size):

    kernel = np.ones((size,size))
    kernel = kernel/(size**2)
    filtered_img = convolve2d(img, kernel, mode='same', boundary='symm')
    return filtered_img

def otsu_mask(img):
    thresh = filters.threshold_otsu(img)
    img_copy = np.zeros(np.shape(img))
    img_copy[img > thresh] = 1

    return img_copy
     
def generate_matrix(size, width):
    matrices = []
    for w in width:
        matrix_horizontal = np.zeros((size, size))
        matrix_horizontal[int(size/2-w/2):int(size/2+w/2), :] = 1

        matrix_vertical = np.zeros((size, size))
        matrix_vertical[:, int(size/2-w/2):int(size/2+w/2)] = 1

        matrix_diag_up = np.zeros((size, size))
        for i in range(size):
            matrix_diag_up[i:i+w,i: i+w] = 1

        matrix_diag_down = np.zeros((size, size))
        for i in range(size - w + 1):
            matrix_diag_down[i:i+w, size-i-w:size-i] = 1

        matrices.extend([matrix_horizontal, matrix_vertical, matrix_diag_up, matrix_diag_down])

    # Chessboard pattern
    chessboard = np.zeros((size, size))
    chessboard[1::2, ::2] = 1
    chessboard[::2, 1::2] = 1

    matrices.append(chessboard)

    # Ensure the total sum of pixels is 0
    for matrix in matrices:
        matrix[matrix == 0] = -1
        matrix[matrix == 1] = 1 / np.sum(matrix[matrix == 1])
        matrix[matrix == -1] = -1 / ((-1) * np.sum(matrix[matrix == -1]))

    return matrices

def emphatyze_morphology(img, lista_k):
    k_size = np.shape(lista_k[0])
    img_size = np.shape(img)
    img_result = np.zeros(img_size)

    for i in range(0, img_size[0], k_size[0]):
        for j in range(0, img_size[1], k_size[1]):
            img_patch = img[i:i+k_size[0], j:j+k_size[1]]
            img_to_add = convolution_with_max_density_optic(img_patch, lista_k)
            img_to_add[img_to_add <0] = 0
            img_to_add[img_to_add > 1 ] =1
            img_result[i:i+k_size[0], j:j+k_size[1]] = img_to_add

    return img_result

def enchance_contrast(img,f):

    media = np.mean(img)
    diff = img - np.array(media)
    img_con = img + f*diff
    img_con[img_con < 0] = 0
    img_con[img_con > 1] = 1

    return img_con

def local_contrast_adjustment(image, binary_mask, factor=1.0):
    """
    Regola il contrasto localmente sull'immagine utilizzando la differenza rispetto alla media,
    sulla regione specificata dalla maschera binaria.

    Parameters:
    - image: np.array
        L'immagine di input.
    - binary_mask: np.array, dtype=bool
        La maschera binaria che indica la regione sulla quale applicare l'aggiustamento del contrasto.
    - factor: float, optional
        Fattore di regolazione del contrasto. Un valore superiore a 1 aumenterà il contrasto,
        mentre un valore inferiore a 1 lo diminuirà.

    Returns:
    - np.array
        L'immagine con il contrasto regolato localmente sulla regione specificata dalla maschera.
    """
    # Copia l'immagine originale per mantenere l'originale inalterata
    image_copy = image.copy()

    # Estrai la regione dalla maschera
    region_of_interest = image_copy[binary_mask]

    # Calcola la media della regione di interesse
    mean_intensity = np.mean(region_of_interest)

    # Calcola la matrice delle differenze rispetto alla media
    difference_matrix = region_of_interest - mean_intensity

    # Regola il contrasto moltiplicando le differenze per il fattore di regolazione
    adjusted_differences = factor * difference_matrix

    # Sostituisci la regione originale con quella regolata nella copia dell'immagine
    image_copy[binary_mask] = mean_intensity + adjusted_differences

    image_copy[image_copy <0] = 0
    image_copy[image_copy>1] = 1

    return image_copy

def local_gamma_correction(image, binary_mask, gamma=1.0):
    """
    Applica correzione gamma localmente sull'immagine sulla regione specificata dalla maschera binaria.

    Parameters:
    - image: np.array
        L'immagine di input.
    - binary_mask: np.array, dtype=bool
        La maschera binaria che indica la regione sulla quale applicare la correzione gamma.
    - gamma: float, optional
        Fattore gamma per la correzione gamma. Un valore superiore a 1 aumenta il contrasto,
        mentre un valore inferiore a 1 lo diminuisce.

    Returns:
    - np.array
        L'immagine con la correzione gamma applicata localmente sulla regione specificata dalla maschera.
    """
    # Copia l'immagine originale per mantenere l'originale inalterata
    image_copy = image.copy()

    # Estrai la regione dalla maschera
    region_of_interest = image_copy[binary_mask]

    # Applica la correzione gamma solo sulla regione di interesse
    region_of_interest_corrected = region_of_interest ** gamma

    # Sostituisci la regione originale con quella corretta nella copia dell'immagine
    image_copy[binary_mask] = region_of_interest_corrected

    return image_copy



def metti_toppe(img_g, range_min = 0.1, range_max = 0.7):
    img_f = low_pass(img_g,16)
    img_f = enchance_contrast(img_f,0.2)
    img_gamma = img_f ** 1.5
    #
    ot = otsu_mask(img_gamma)
    selem = np.ones((32, 64))  
    closing = ndimage.morphology.binary_closing(ot, structure=selem)
    image_no_a = equalize_within_mask(img_g, closing, range_min,range_max)
    img_p = image_no_a
    return img_p


In [14]:
for img in tqdm(images):
    image_path = os.path.join(path_im, img)
    img_o = mpimg.imread(image_path)
    img_pre = img_o.copy()
    img_ar=np.array(img_pre)
    img_s = img_ar[:,:,0]
    img_s = exposure.equalize_hist(img_s)
    img_s = low_pass(img_s,24)
    img_s = img_s ** 60
    img_s = img_s - 0.2
    img_s[img_s < 0] = 0
    img_g = img_ar[:,:,1]-0.05*img_s
    imig_g = exposure.equalize_hist(img_g)
    img_d = img_g #- 0.1*img_s

    mask_cerchio = img_d < 0.00784 
    img_d[mask_cerchio] = 1.0
    img_e = equalize_within_mask(img_d, ~mask_cerchio, range_min=0, range_max=1)
    
    if np.mean(img_e[~mask_cerchio]) < 0.35:
        img_gamma = (img_e) **(0.1+np.mean(img_e[~mask_cerchio]))
    elif np.mean(img_e[~mask_cerchio]) > 0.55:
        img_gamma = (img_e) ** (0.7+np.mean(img_e[~mask_cerchio]))
    else:
        img_gamma = img_e.copy()

    img_l = low_pass(img_gamma,8)
    img_gamma = img_l ** 0.6
    img_con = enchance_contrast(img_gamma,1)
    img_ad = exposure.equalize_adapthist(img_con, np.shape(img_g)[0]/8)
    img_8 = (img_ad*255).astype(np.uint8)
    img_to_save = Image.fromarray(img_8)
    img_to_save.save(os.path.join(path_result,img))


100%|██████████| 600/600 [2:44:25<00:00, 16.44s/it]  


In [13]:
np.mean(img_e[~mask_cerchio])

0.16040393174615503